In [189]:
#  Importing Necessary Library
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from random import random
from random import randrange
from sklearn.metrics import accuracy_score
from math import exp
from scipy.special import softmax

In [67]:
df=pd.read_csv("dataset/iris.csv")

In [256]:
# Load a CSV file
def loadCsv(filename):
        trainset=pd.read_csv(filename)
        return trainset

# Normalization

def normalize(dataset):
    min_max_obj=MinMaxScaler()
    transformed_data=min_max_obj.fit_transform(dataset)
    return transformed_data

# Convert column values to float
def column_to_float(dataset, column):
        return dataset[column].astype(float)

#  Label encoding target variable

def encode_target(dataset,target_column):
        label_encoder=LabelEncoder()
        label_encoded=label_encoder.fit_transform(dataset.iloc[:,target_column])
        with open("artifacts/label_encoder.pickle","wb") as s:
                pickle.dump(label_encoder,s)
        return label_encoded

# Method to create cross validation splits

def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = dataset.to_numpy().tolist()
        fold_size = int(len(dataset_copy) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split

#  Method to Create a neural network

# Creating Neural network with 1 hidden layes
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        #print(network)
        # initialize parameters of activation function g(x) = k0 + k1x
        k=[random(),random()]
        return network,k


# Calculate neuron activation for an input
def activate(weights, inputs):
        activation = weights[-1]   # Weights[-1]  is Bias term
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation


# Transfer neuron activation     g(x) = k0 + k1x
def transfer(activation,K):
        return K[0]+K[1]*(activation)


# Forward propagate input to a network output  -  Hidden Layers use Ada Acti Activation (g(x) = k0 + k1x) and output layer use Softmax
def forward_propagate(network, row,K):
        inputs = row
        for index,layer in enumerate(network):
                if index!=len(network)-1:
                        new_inputs = []
                        for neuron in layer:
                                activation = activate(neuron['weights'], inputs)
                                print(f"Hidden Layer {index} ,Applying Ada Acti Activation")
                                neuron['output'] = transfer(activation,K)
                                new_inputs.append(neuron['output'])
                        inputs = new_inputs
                        # print(f"End of a layer{index}",inputs)
                else:
                        print("Final output layer : Applying Softmax")
                        i=1
                        new_inputs = []
                        for neuron in layer:
                                activation = activate(neuron['weights'], inputs)
                                # print("Activation value",activation)
                                neuron['output'] = activation
                                new_inputs.append(neuron['output'])
                        # print(new_inputs)
                        # inputs=new_inputs
                        inputs = softmax(new_inputs,axis=0)   
                        # print("After applying softmax",inputs)                     

                        
        return inputs


# Calculate the derivative of an neuron output
def transfer_derivative(output):
        return output * (1.0 - output)


# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
        for i in reversed(range(len(network))):
                layer = network[i]
                errors = list()
                if i != len(network)-1:
                        for j in range(len(layer)):
                                error = 0.0
                                for neuron in network[i + 1]:
                                        error += (neuron['weights'][j] * neuron['delta'])
                                errors.append(error)
                else:
                        for j in range(len(layer)):
                                neuron = layer[j]
                                errors.append(expected[j] - neuron['output'])
                for j in range(len(layer)):
                        neuron = layer[j]
                        neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [267]:
net,k=initialize_network(4,3,3)

# print(forward_propagate(net,df.iloc[1,:],k))


In [70]:
#  converting Independent variables into float

for col in df.columns[:-1]:
    df[col]=column_to_float(df,col)

In [71]:
# Encoding  dependent variable 

df.iloc[:,-1]=encode_target(df,-1)

# df.iloc[:,-1]

In [76]:
df.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0


In [172]:
for f in [[1,2,3],[4,5,6],[11,22,33]]:
    h=list([[1,2,3],[4,5,6],[11,22,33]])
    # print(h)
    h.remove(f)
    print(sum(h,[]))

[4, 5, 6, 11, 22, 33]
[1, 2, 3, 11, 22, 33]
[1, 2, 3, 4, 5, 6]


In [254]:
 np.random.randn(2, 3)

array([[ 1.35022802, -1.70163072, -0.33832218],
       [-0.10627487, -0.11439432, -0.36702633]])